This notebook will get you started on training a logistic regression model to predict heart disease using historic data. This is a classic exercise from Hastie's *Elements of statistical learning*. I also used parts of the notebook: 
https://github.com/empathy87/The-Elements-of-Statistical-Learning-Python-Notebooks/blob/master/examples/South%20African%20Heart%20Disease.ipynb

in creating this exercise.

In [279]:
import numpy as np
from numpy.linalg import norm
from sklearn.preprocessing import normalize
import pandas as pd 
import matplotlib.pyplot as plt

In [280]:
# Read the data in. Will need an internet connection!
data = pd.read_csv('http://www-stat.stanford.edu/~tibs/ElemStatLearn/datasets/SAheart.data', error_bad_lines=False)
# More info on data can be found at: https://web.stanford.edu/~hastie/ElemStatLearn/datasets/SAheart.info.txt

In [281]:
# View the first rows. Note that the data points are encoded as rows, with sbp--age being the
# components of x and chd (= chronic heart disease) being the response variable, y.
data.head()
target = 'chd'
features = ['sbp', 'tobacco', 'ldl', 'famhist', 'obesity', 'alcohol', 'age']
normalize_features = ['sbp', 'tobacco', 'ldl', 'obesity', 'alcohol', 'age']

In [282]:
# The following command Replaces "Present" with 1 and "Absent" with 0 in the "famhist" column
data['famhist'] = pd.get_dummies(data['famhist'])['Present']
data.head()

,row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,1,160,12.00,5.73,23.11,1,49,25.30,97.20,52,1
1,2,144,0.01,4.41,28.61,0,55,28.87,2.06,63,1
2,3,118,0.08,3.48,32.28,1,52,29.14,3.81,46,0
3,4,170,7.50,6.41,38.03,1,51,31.99,24.26,58,1
4,5,134,13.60,3.50,27.78,1,60,25.99,57.34,49,1


In [283]:
# Extract the data into numpy arrays
y =  data[target].values

# Normalize data
X_temp0 = normalize(data[normalize_features].values, axis = 0) # normalizes all cols we want to normalize
X_temp1 = np.column_stack((X_temp0[:,0:3], data['famhist'].values)) # append first part of X_normalized cols with famhist
X = np.column_stack((X_temp1, X_temp0[:,3:6])) # append second part of X_normalized

data1 = pd.DataFrame(X, columns = features)
data1.head()

,sbp,tobacco,ldl,famhist,obesity,alcohol,age
0,0.053234,0.095371,0.051543,1.0,0.044616,0.151708,0.053482
1,0.047910,0.000079,0.039669,0.0,0.050912,0.003215,0.064796
2,0.039260,0.000636,0.031304,1.0,0.051388,0.005947,0.047311
3,0.056561,0.059607,0.057660,1.0,0.056414,0.037864,0.059654
4,0.044583,0.108087,0.031484,1.0,0.045833,0.089495,0.050397


Define the sigmoid functoin and then apply the following more convenient notation:
\begin{align}
h_{\mathbf{\theta},b}(\mathbf{x}) := \sigma(\boldsymbol{\theta}^{\top}\mathbf{x} + b)
\end{align}

In [284]:
# Define sigmoid function
def sigma(z):
    output = 1.0 / (1.0 + np.exp(-z))
    
    # make a boundary to deal with machine percision
    smallest = .00001
    largest = .9999
    
    output = max(output, smallest)
    output = min(output, largest)
    
    return output

def h(theta,b,x):
    return sigma(np.dot(theta,x) + b)

In [293]:
## Insert your logistic regression code here.
def TrainingLogisticRegression(theta_0, b_0, X, y, alpha, K_max):
    # Function for training logistic regression model
    # Inputs:
        # theta_0,b_0 : (random) initializations for parameters
        # X,y : labeled training data
        # alpha: step size/ learning rate
        # K_max: max number of iterations.
    theta = np.squeeze(theta_0)
    b = b_0
    N = X.shape[0]
    d = X.shape[1]
    
    loss_function_trajectory = np.zeros([K_max,1])
    
    for k in range(K_max):
        theta_grad = 0
        b_grad = 0
        loss_function_value = 0
        
        for i in range(N):
            theta_grad += -(y[i] - h(theta, b, X[i,:])) * X[i,:]
            b_grad += -y[i] + h(theta, b, X[i,:])
            loss_function_value += -y[i] * np.log(h(theta, b, X[i,:])) - (1 - y[i]) * np.log(1 - h(theta, b, X[i,:])) 
        
        theta -= alpha * theta_grad
        b -= alpha * b_grad
        loss_function_trajectory[k] = loss_function_value
        
        if k % 50 == 0:
            print(loss_function_value)
            
    return theta, b, loss_function_trajectory

In [299]:
# Initializing parameters
## Split your data into a train set and a test set here. 
# You should have 80 data points in your test set.
All_indices = range(0,462) 
Train_indices = np.random.choice(All_indices,size=382,replace=False)
Test_indices = np.setdiff1d(All_indices,Train_indices)

X_train = X[Train_indices,:]
y_train = y[Train_indices]
#print(X_train)

X_test = X[Test_indices,:]
y_test = y[Test_indices]

theta_0 = np.random.randn(len(features),1)
b_0 = np.random.randn(1)
alpha = .00001
K_max = 500

# training
theta, b, loss_function_trajectory = TrainingLogisticRegression(theta_0, b_0, X_train, y_train, alpha, K_max)

[349.72874287]
[346.07621565]
[342.55439472]
[339.16002124]
[335.88978339]
[332.74032376]
[329.70824677]
[326.79012617]
[323.98251253]
[321.2819409]


In [300]:
def Classifier(theta,b,x):
    probability = h(theta,b,x) # = sigmoid(theta^{T}x - b)
    classification = np.round(probability) # if probability >= 0.5 then predict y = 1. if prob < 0.5 then predict y = 0
    return probability,classification

In [301]:
total = 0

for i in range(0, 80):
    probability, classification = Classifier(theta, b, X_test[i,:])
    if classification == y_test[i]:
        total += 1

print(total / 80)

0.6625
